In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import scipy.stats as stats

In [4]:
# DDI in DisGeNET，UMLS
ori_DDA_df = pd.read_csv("disease_to_disease_ALL_extract.csv",index_col=0)
print(ori_DDA_df.shape)
ori_DDA_df[:2]

/home/yu/anaconda3/envs/jxx/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(63680896, 3)


,diseaseId1_name,diseaseId2_name,jaccard_genesPlusVariant
0,Abdominal Pain,"Abdomen, Acute",0.003300
1,Acne Vulgaris,"Abdomen, Acute",0.010526


In [50]:
# ori_DDA_df.to_csv("disease_to_disease_ALL_extract.csv")

In [5]:
sym_fea = pd.read_csv("2020_TF-IDF_dropdup.csv",index_col=0)
sym_fea[:2]

,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
"Nevus, Intradermal",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Superinfection,0.0,0.0,0.0,0.635781,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.309646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# UMLS-mesh
mapping_df = pd.read_csv("DisGeNET-disease_mappings.tsv",sep="\t")
mapping_df = mapping_df[mapping_df["vocabulary"]=="MSH"]
mapping_df.reset_index(drop=True,inplace=True)
mapping_df[:2]

,diseaseId,name,vocabulary,code,vocabularyName
0,C1970109,AROMATASE EXCESS SYNDROME,MSH,C000591739,Aromatase Excess Syndrome
1,C1970109,AROMATASE EXCESS SYNDROME,MSH,C000591739,"Familial gynecomastia, due to increased aromat..."


#### UMLS->MSH name
- DisGeNET的DDA数据中除MeSH heading外还包含了MeSH Supplementary ---> 30264个dis
    - 30264--->29327 个在id映射文件中 --->5366 个umls name转为mesh name
    -(多个umls name可能对应一个mesh name)

In [7]:
# 1、Get Unique Diseases in DDA Data
DDA_dis = pd.concat([pd.DataFrame({"dis":ori_DDA_df["diseaseId1_name"]}),pd.DataFrame({"dis":ori_DDA_df["diseaseId2_name"]})])
print(len(DDA_dis)) 

DDA_dis.drop_duplicates(inplace=True)
print(len(DDA_dis)) 

DDA_dis.reset_index(drop=True,inplace=True)
DDA_dis[:2]

127361792
30264


,dis
0,Abdominal Pain
1,Acne Vulgaris


In [8]:
# 2、A UMLS name corresponds to multiple vocabularyNames
tar_dis_ls = sym_fea.index
tar_dis_df = pd.DataFrame(tar_dis_ls,columns=["dis"])
print(tar_dis_df.shape)
print(tar_dis_df[:5])

join_df = pd.merge(DDA_dis,mapping_df,how='inner',left_on="dis",right_on="name")
print(join_df.shape)
join_df.head()

# join_df.to_csv("join_df.csv")

(4268, 1)
                                dis
0                Nevus, Intradermal
1                    Superinfection
2                          Coloboma
3  Abnormalities, Radiation-Induced
4         Bronchial Hyperreactivity
(29327, 6)


,dis,diseaseId,name,vocabulary,code,vocabularyName
0,Abdominal Pain,C0000737,Abdominal Pain,MSH,D015746,Abdominal Pain
1,Abdominal Pain,C0000737,Abdominal Pain,MSH,D015746,Colicky Pain
2,Acne Vulgaris,C0001144,Acne Vulgaris,MSH,D000152,Acne Vulgaris
3,Acne Vulgaris,C0001144,Acne Vulgaris,MSH,D000152,Acne
4,Adenitis,C0001416,Adenitis,MSH,D008199,Adenitis


In [9]:
# 3、UMLS name --->Mesh name
trans = {}

for i in tqdm(range(len(join_df))):
    if join_df.loc[i,"vocabularyName"] in tar_dis_ls:
        UMLS_name = join_df.loc[i,"dis"]
        mesh_name = join_df.loc[i,"vocabularyName"]
        
        trans[UMLS_name]= mesh_name  
            
print(len(trans)) 

100%|██████████| 29327/29327 [00:00<00:00, 132046.66it/s]

5366


In [10]:
trans_df = pd.DataFrame.from_dict(trans,orient="index")
trans_df.reset_index(inplace=True)
trans_df.columns=["UMLS name","MSH name"]
trans_df[:2]

,UMLS name,MSH name
0,Acne Vulgaris,Acne Vulgaris
1,Adenitis,Lymphadenitis


In [20]:
# trans_df.to_csv("trans_df.csv")

#### ->MSH name

In [11]:
DDA_mesh_arr = []  

sour_umls_name =list(trans_df["UMLS name"])

for ite in tqdm(range(len(ori_DDA_df))):
    dis1 = ori_DDA_df.loc[ite,"diseaseId1_name"]
    dis2 = ori_DDA_df.loc[ite,"diseaseId2_name"]
    
    if (dis1 in sour_umls_name) and (dis2 in sour_umls_name):
        jac = ori_DDA_df.loc[ite,"jaccard_genesPlusVariant"]
        DDA_mesh_arr.append([trans[dis1],trans[dis2],jac])
        
print(len(DDA_mesh_arr)) 

DDA_mesh_df = pd.DataFrame(DDA_mesh_arr,columns=["diseaseId1_name","diseaseId2_name","jaccard"])
DDA_mesh_df.to_csv("DDA_mesh_df.csv")
DDA_mesh_df.head()

100%|██████████| 63680896/63680896 [1:17:35<00:00, 13678.22it/s]


8876054


,diseaseId1_name,diseaseId2_name,jaccard
0,Alzheimer Disease,Colic,0.000294
1,Anaplasia,Colic,0.001859
2,Arteriosclerosis,Colic,0.000499
3,Asthma,Colic,0.000477
4,Atherosclerosis,Colic,0.000489


In [19]:
# DDA_mesh_df.to_csv("DDA_mesh_df.csv")

### Extract 5366 UMLS diseases and  corresponding model prediction results

In [14]:
predict_score = pd.read_csv("model_predictAll_result.csv",index_col=0)

dis_index_inSym = []
sym_fea_index_lower = [s.lower() for s in sym_fea.index]

for dis in tqdm(trans_df["MSH name"]):
    index_inSym = sym_fea_index_lower.index(dis.lower())
    dis_index_inSym.append(index_inSym)
    
print(len(dis_index_inSym))
dis_index_inSym[:5]

/home/yu/anaconda3/envs/jxx/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
100%|██████████| 5366/5366 [00:00<00:00, 56785.04it/s]

5366


[1279, 3131, 2883, 151, 2921]